In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import warnings
warnings.filterwarnings('ignore')

# YoloV5

In [4]:
import torch
from IPython.display import Image
import shutil
import os
from random import choice

In [5]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15291, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15291 (delta 0), reused 1 (delta 0), pack-reused 15287
Receiving objects: 100% (15291/15291), 14.19 MiB | 23.37 MiB/s, done.
Resolving deltas: 100% (10483/10483), done.


In [6]:
%cd /content/yolov5/

/content/yolov5


In [7]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.5.1 which is incompatible.


In [8]:
# arrays to store file names
imgs = []
xmls = []

# setup directory names
train_path = "/content/drive/MyDrive/pothole/yolo/train/images"
val_path = "/content/drive/MyDrive/pothole/yolo/valid/images"
crs_path = "/content/drive/MyDrive/pothole/yolo/images"

train_ratio = 0.8
val_ratio = 0.2

# total count of imgs
total_img_count = len(os.listdir(crs_path))/2

# sorting count of imgs
for (dirname,dirs, files) in os.walk(crs_path):
#     print(files)
    for filename in files:
#         print("hello")
        if filename.endswith(".txt"):
            xmls.append(filename)
        else:
            imgs.append(filename)

# counting range for cycles
count_for_train = int(len(imgs)*train_ratio)
count_for_val = int(len(imgs)*val_ratio)
print("training images are : ",count_for_train)
print("validation images are : ",count_for_val)

training images are :  59
validation images are :  14


In [9]:
len(imgs)

74

In [10]:
train_image_path = "/content/drive/MyDrive/pothole/yolo/train/images"
train_label_path = "/content/drive/MyDrive/pothole/yolo/train/labels"
val_image_path = "/content/drive/MyDrive/pothole/yolo/valid/images"
val_label_path = "/content/drive/MyDrive/pothole/yolo/valid/labels"

if not os.path.isdir(train_image_path):
    os.makedirs(train_image_path)
if not os.path.isdir(train_label_path):
    os.makedirs(train_label_path)
if not os.path.isdir(val_image_path):
    os.makedirs(val_image_path)
if not os.path.isdir(val_label_path):
    os.makedirs(val_label_path)

# cycle for train dir
for x in range(count_for_train):
    file_jpg = choice(imgs)
    file_xml = file_jpg[:-4] + ".txt"
    
    shutil.copy(os.path.join(crs_path,file_jpg),os.path.join(train_image_path,file_jpg))
    shutil.copy(os.path.join(crs_path,file_xml),os.path.join(train_label_path,file_xml))
    
    imgs.remove(file_jpg)
    xmls.remove(file_xml)
    
# cycle for test dir
for x in range(count_for_val):
    file_jpg = choice(imgs)
    file_xml = file_jpg[:-4] + ".txt"
    
    shutil.copy(os.path.join(crs_path,file_jpg),os.path.join(val_image_path,file_jpg))
    shutil.copy(os.path.join(crs_path,file_xml),os.path.join(val_label_path,file_xml))
    
    imgs.remove(file_jpg)
    xmls.remove(file_xml)

# rest of files
print("images length - ",len(imgs))
print(imgs,xmls)
for x in imgs:
    file_jpg = x
    file_xml = file_jpg[:-4] + ".txt"
    
    shutil.copy(os.path.join(crs_path,file_jpg),os.path.join(val_image_path,file_jpg))
    shutil.copy(os.path.join(crs_path,file_xml),os.path.join(val_label_path,file_xml))
# shutil.copytree(crs_path,val_path)

images length -  1
['po_578_jpg.rf.9a02fe4378b979a854c2420c82975862.jpg'] ['po_578_jpg.rf.9a02fe4378b979a854c2420c82975862.txt']


In [11]:
with open(r'/content/drive/MyDrive/pothole/yolo/data.yaml', 'r') as file:
    lines = file.readlines()
    print(lines)

['train: /content/drive/MyDrive/pothole/yolo/train/images\n', 'val: /content/drive/MyDrive/pothole/yolo/valid/images\n', '\n', '\n', 'nc: 2\n', "names: ['manhole', 'pothole']\n", '\n', 'roboflow:\n', '  workspace: tejas-veer\n', '  project: potholedata3\n', '  version: 1\n', '  license: CC BY 4.0\n', '  url: https://universe.roboflow.com/tejas-veer/potholedata3/dataset/1']


In [14]:
#!wandb disabled
!python train.py --img 415 --batch 16 --epochs 5 --data /content/drive/MyDrive/pothole/yolo/data.yaml --weights yolov5s.pt --cache --workers 2

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/pothole/yolo/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=16, imgsz=415, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-117-g85f6019 Python-3.8.10 torch-1.13.1+cu116 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv

In [15]:
!zip -r /content/drive/MyDrive/pothole/file.zip /content/yolov5

  adding: content/yolov5/ (stored 0%)
  adding: content/yolov5/.git/ (stored 0%)
  adding: content/yolov5/.git/description (deflated 14%)
  adding: content/yolov5/.git/info/ (stored 0%)
  adding: content/yolov5/.git/info/exclude (deflated 28%)
  adding: content/yolov5/.git/hooks/ (stored 0%)
  adding: content/yolov5/.git/hooks/applypatch-msg.sample (deflated 42%)
  adding: content/yolov5/.git/hooks/update.sample (deflated 68%)
  adding: content/yolov5/.git/hooks/pre-merge-commit.sample (deflated 39%)
  adding: content/yolov5/.git/hooks/fsmonitor-watchman.sample (deflated 52%)
  adding: content/yolov5/.git/hooks/pre-commit.sample (deflated 45%)
  adding: content/yolov5/.git/hooks/prepare-commit-msg.sample (deflated 50%)
  adding: content/yolov5/.git/hooks/pre-rebase.sample (deflated 59%)
  adding: content/yolov5/.git/hooks/commit-msg.sample (deflated 44%)
  adding: content/yolov5/.git/hooks/post-update.sample (deflated 27%)
  adding: content/yolov5/.git/hooks/pre-receive.sample (deflate